In [1]:
from google.colab import drive
drive.mount('/content/drive')

from keras.models import Model
from keras import backend as K
from keras.callbacks import ModelCheckpoint

from keras.layers import Input, concatenate, Conv2D, MaxPooling2D 
from keras.layers import UpSampling2D, Convolution2D, ZeroPadding2D
from keras.optimizers import Adam
from keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D, BatchNormalization, Add
from keras.layers.core import Activation


import numpy as np
from numpy import genfromtxt
from numpy.testing import assert_allclose
from scipy import misc
import matplotlib.pyplot as plt

import os
from tqdm import tqdm
from skimage.transform import resize
from skimage.io import imsave
from PIL import Image


def dice(y_true, y_pred):                                  # Метрика сходства двух изображений
    y_true_f = K.flatten(y_true[:,:,:,0])
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 0.001) / (K.sum(y_true_f) + K.sum(y_pred_f) + 0.001)

def dice_border(y_true, y_pred):
  y_true_f = K.flatten(y_true[:,:,:,1])
  y_pred_f = K.flatten(y_pred*y_true[:,:,:,1])
  intersection = K.sum(y_true_f * y_pred_f)
  return (2. * intersection + 0.001) / (K.sum(y_true_f) + K.sum(y_pred_f) + 0.001)

def dice_coef(y_true, y_pred):                            
    return (dice(y_true, y_pred) + dice_border(y_true, y_pred)) /2 

def dice_coef_loss(y_true, y_pred):                        # Функция потерь для минимизации
    return -dice_coef(y_true, y_pred)

  
#  def dice_coef(y_true, y_pred):
#  
#    y_true_f = K.flatten(y_true[:,:,:,0])
#    y_pred_f = K.flatten(y_pred)
#    intersection = K.sum(y_true_f * y_pred_f)
#    return (2. * intersection + 0.001) / (K.sum(y_true_f) + K.sum(y_pred_f) + 0.001)
#
#
#def dice_coef_loss(y_true, y_pred):
#    return -dice_coef(y_true, y_pred)

def Conv2D_34skipblock(filters, strides=(3, 3), activation="relu", padding="same"):
    assert(isinstance(strides, tuple))
    assert(2 == len(strides))

    def f(x):
        conv1 = Conv2D(filters, strides, padding=padding)(x)
        batch_norm1 = BatchNormalization()(conv1)
        act1 = Activation(activation=activation)(batch_norm1)
        conv2 = Conv2D(filters, strides, padding=padding)(act1)
        batch_norm2 = BatchNormalization()(conv2)
        act2 = Activation(activation=activation)(batch_norm2)
        add = Add()([x, act2])
        return add
    return f

def uresnet(skip_block, image_rows, image_cols, img_channels=1):

    inputs = Input((image_rows, image_cols, img_channels))
    conv0 = Conv2D(32, (3, 3), padding='same')(inputs)
    batch_norm0 = BatchNormalization()(conv0)
    act0 = Activation(activation='relu')(batch_norm0)

    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(act0)
    conv1 = skip_block(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = skip_block(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = skip_block(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = skip_block(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = skip_block(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = skip_block(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = skip_block(256, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = skip_block(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = skip_block(512, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = skip_block(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = skip_block(256, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = skip_block(256, (3, 3), activation='relu', padding='same')(conv6)
    
    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = skip_block(128, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = skip_block(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = skip_block(64, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = skip_block(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = skip_block(32, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = skip_block(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])
        
    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

    return model

Mounted at /content/drive


Using TensorFlow backend.


In [2]:
K.set_image_data_format('channels_last')

NB_EPOCH = 40
BATCH_SIZE = 1
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIMIZER = Adam()
IMG_ROWS, IMG_COLS, IMG_CHANNELS = 384, 384, 1
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, IMG_CHANNELS)
layer = '0'
TRAIN = 512
PATH = '/content/drive/My Drive/Rosneft Seismic/train/'
x_train = []
y_train = []
y_mask  = []
print('LOAD TRAIN DATA')
for i in tqdm(range(TRAIN)):
  img = np.array(Image.open(PATH+'images/inline_'+str(i+3160)+'.png'))
  img = img.astype('float32')
  img = (img[:, :, 0]+img[:, :, 1]+img[:, :, 2])/3
  img -= img.mean()
  img /= img.std()
  x_train.append(img)
  y_train.append(np.array(Image.open(PATH+'answer/'+layer+'/inline_'+str(i+3160)+'.png')))
  y_mask.append(np.array(Image.open(PATH+'answer/'+layer+'_border/inline_'+str(i+3160)+'.png')))

print('END LOAD TRAIN DATA') 
print('STAR PREPROCCECING')
x_train = np.array(x_train)
x_train = x_train[:, :, :, np.newaxis]

y_train = np.array(y_train)
y_train = y_train[:, :, :, np.newaxis, np.newaxis]
y_mask = np.array(y_mask)
y_mask = y_mask[:, :, :, np.newaxis, np.newaxis]
y_train = np.concatenate((y_train, y_mask), axis = 3)

y_train = y_train.astype('float32')
y_train /= 255



print('END PREPROCCECING')

  0%|          | 0/512 [00:00<?, ?it/s]

LOAD TRAIN DATA


100%|██████████| 512/512 [03:23<00:00,  2.69it/s]


END LOAD TRAIN DATA
STAR PREPROCCECING
END PREPROCCECING


In [37]:
y_train.shape

(512, 384, 384, 2, 1)

In [3]:
model = uresnet(Conv2D_34skipblock, IMG_ROWS, IMG_COLS, IMG_CHANNELS)

model_checkpoint = ModelCheckpoint('weights_layer_0_v1.h5', monitor='val_loss', save_best_only=True)

history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, verbose=VERBOSE, 
                    shuffle=True, validation_split=VALIDATION_SPLIT, callbacks=[model_checkpoint])

plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('TRAIN RESULT')
plt.ylabel('dice_coef')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc = 'upper left')
plt.grid()
plt.rcParams["figure.figsize"] = [100,100]
plt.show()

















Train on 409 samples, validate on 103 samples
Epoch 1/40
409/409 [==============================] - 239s 585ms/step - loss: -0.8931 - dice_coef: 0.8931 - val_loss: -0.9759 - val_dice_coef: 0.9759
Epoch 2/40
 42/409 [==>...........................] - ETA: 3:06 - loss: -0.9818 - dice_coef: 0.9818

KeyboardInterrupt: ignored